In [ ]:
!pip install yt_dlp TTS

In [ ]:
!pip3 install python-dotenv

In [ ]:
!pip install openai==0.28.1

In [ ]:
!pip install ffmpeg-python

## 영상, 음성 다운로드

In [ ]:
import yt_dlp

def url_to_mp3(url, output_path):
    # 다운로드 옵션 설정
    ydl_opts = {
        "format": "bestaudio/best",  # 최적의 오디오 품질로 다운로드
        "outtmpl": output_path,  # 출력 경로 및 파일명 설정
        "postprocessors": [{  # 후처리 옵션 설정 (오디오 포맷 변경)
            "key": "FFmpegExtractAudio",  # 오디오 추출을 위한 후처리
            "preferredcodec": "mp3",  # MP3 포맷으로 변환
            "preferredquality": "192"  # MP3 품질 192kbps로 설정
        }]
    }

    # yt-dlp로 다운로드 실행
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

def url_to_mp4(url, output_path):
    """비디오+오디오 다운로드 (MP4 포맷)"""
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
        'outtmpl': output_path,
        'merge_output_format': 'mp4'  # ✅ 포맷 강제 지정
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    print(f"🎥 비디오 저장 완료: {output_path.replace('%(ext)s', 'mp4')}")



## 음성 → 텍스트 변환


#### 출력 파일 생성
- whisper_response.json: 원본 API 응답 데이터 (모든 메타데이터 포함)

- full_transcription.txt: 전체 변환 텍스트 (연속된 문자열)

- segments_timestamps.txt: 문장별 시작/종료 시간 기록

In [ ]:
import openai
import json

def mp3_to_text(file_path, api_key):
    openai.api_key = api_key

    with open(file_path, "rb") as audio_file:
        response = openai.Audio.transcribe(
            file=audio_file,
            model="whisper-1",
            language="ko",  # 한국어 음성 처리
            response_format="verbose_json",
            timestamp_granularities=["segment"]
        )

    # JSON 응답 저장
    with open("whisper_response.json", "w", encoding="UTF-8") as file:
        json.dump(response, file, ensure_ascii=False, indent=2)

    # 전체 텍스트 저장
    with open("full_transcription.txt", "w", encoding="UTF-8") as file:
        file.write(response['text'])

    # 세그먼트 및 타임스탬프 저장
    with open("segments_timestamps.txt", "w", encoding="UTF-8") as file:
        for segment in response['segments']:
            line = f"{segment['text']} - 시작: {segment['start']:.2f}, 종료: {segment['end']:.2f}\n"
            file.write(line)

##  영상 스크립트에서 쇼츠 포인트를 찾아 저장

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key=os.getenv("OPEN_AI_KEY")

In [ ]:
import openai

def text_to_shorts_points(full_text_path, api_key):
    openai.api_key = api_key  # <- 최신 SDK에서는 이렇게 설정

    # ✅ 텍스트 파일 열어서 내용 읽기
    with open(full_text_path, "r", encoding="UTF-8") as file:
        full_text = file.read()

    # 2. 텍스트를 기반으로 ChatGPT에게 쇼츠 포인트 요청
    chat_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "너는 최고의 유튜브 편집자이자 교육 전문가야."},
            {"role": "user", "content": f"""
            [규칙]
            - 주어지는 스크립트에서 공부 중 꼭 알아야 할 핵심 개념, 설명, 예시, 꿀팁 등을 포함한
              '자연스러운 문단 또는 연속된 문장 덩어리' 단위로 묶어서 쇼츠화에 적합한 포인트를 선별해줘.
            - 학습에 도움이 되게 10문장 이상의 연속된 문장들로 구성된 자연스러운 덩어리여야 해.
            - 각 쇼츠 포인트는 해당 부분을 그대로 복사해서 출력해줘.
            - 문장 순서나 내용을 바꾸지 말고, 반드시 원본 스크립트에서 '연속된 부분'만 추출해줘.
            - 각 쇼츠 포인트 옆에는 '이 부분을 쇼츠화 포인트로 선택한 이유'를 1~2문장으로 설명해줘.

            [출력 형식]
            1. [쇼츠 포인트(여러 문장)] - [선택 이유]
            2. [쇼츠 포인트(여러 문장)] - [선택 이유]

            [스크립트]
            {full_text}
            """}
        ]
    )

    # 3. 결과 저장
    shorts_points = chat_response["choices"][0]["message"]["content"]

    with open("shorts_points.txt", "w", encoding="UTF-8") as file:
        file.write(shorts_points)

    print("✅ 쇼츠 포인트 추출 완료!")


## 1. 세그먼트 읽기 → 2. 쇼츠 포인트 읽기 → 3. 유사도 기반 매칭 → 4. 결과 저장

In [ ]:
from difflib import SequenceMatcher

# --- 1. Whisper 세그먼트 불러오기 ---
def read_segments(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)["segments"]

# --- 2. 쇼츠 포인트 파싱 (여러 줄, 번호, 선택 이유 등 처리) ---
def read_shorts_points(file_path):
    shorts_list = []
    with open(file_path, 'r', encoding='utf-8') as file:
        current_point = []
        for line in file:
            line = line.strip()
            if re.match(r'^\d+\.', line):  # "1.", "2." 등으로 시작
                if current_point:
                    full_text = ' '.join(current_point).split(' - ')[0].strip()
                    shorts_list.append(full_text)
                current_point = [re.sub(r'^\d+\.\s*', '', line)]
            elif line.startswith('- 이 부분을'):  # 선택 이유 라인 무시
                continue
            elif line:
                current_point.append(line)
        if current_point:
            full_text = ' '.join(current_point).split(' - ')[0].strip()
            shorts_list.append(full_text)
    return shorts_list

# --- 3. 세그먼트 윈도우 생성 (연속 n개 묶음) ---
def build_windows(segments, window_size=5):
    windows = []
    for i in range(len(segments) - window_size + 1):
        chunk = segments[i:i + window_size]
        text = ' '.join(seg["text"] for seg in chunk)
        windows.append({
            "text": text,
            "start": chunk[0]["start"],
            "end": chunk[-1]["end"]
        })
    return windows

# --- 4. 유사도 계산 ---
def similarity(a, b):
    a = re.sub(r'\s+', '', a)
    b = re.sub(r'\s+', '', b)
    return SequenceMatcher(None, a, b).ratio()

# --- 5. 쇼츠 포인트와 세그먼트 윈도우 매칭 ---
def match_shorts_to_segments(shorts_points, segments, threshold=0.5, window_size=5):
    windows = build_windows(segments, window_size)
    results = []
    for short in shorts_points:
        best_match = None
        best_score = 0
        short_text = short.strip()
        for window in windows:
            score = similarity(short_text, window["text"])
            if score > best_score:
                best_score = score
                best_match = window
        if best_score >= threshold:
            results.append((short, best_match["start"], best_match["end"]))
        else:
            results.append((short, None, None))
    return results

# --- 6. 결과 파일로 저장 ---
def write_output(file_path, matched):
    with open(file_path, 'w', encoding='utf-8') as file:
        for text, start, end in matched:
            if start is not None:
                file.write(f"{text} - 시작: {start:.2f}, 종료: {end:.2f}\n")
            else:
                file.write(f"{text} - ❌ 매칭 실패\n")

## 영상 자르기

In [ ]:
# fps 추출
import ffmpeg

def get_video_fps(file_path):
    probe = ffmpeg.probe(file_path, v='error', select_streams='v:0', show_entries='stream=r_frame_rate')
    print(probe)
    if not probe['streams']:
      raise ValueError("비디오 스트림을 찾을 수 없습니다. 파일 형식을 확인하세요.")

    fps = probe['streams'][0]['r_frame_rate']
    numerator, denominator = map(float, fps.split('/'))
    return numerator / denominator

## 쇼츠 부분 자르기

In [ ]:
from moviepy.editor import VideoFileClip
import re


def load_timestamps_from_file(text_file):
    shorts_info = []
    # 정규표현식 패턴 업데이트 (텍스트와 타임스탬프 분리)
    pattern = r'^(.*?) - 시작: (\d+\.\d+), 종료: (\d+\.\d+)'

    with open(text_file, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
            match = re.match(pattern, line)
            if match:
                text = match.group(1).strip()
                start_time = float(match.group(2))
                end_time = float(match.group(3))
                shorts_info.append((text, start_time, end_time))
    return shorts_info


def cut_and_save_video(video_path, start_time, end_time, output_path, fps):
    # 동영상 파일 로드
    clip = VideoFileClip(video_path)
    # 주어진 시작과 종료 시간에 따라 클립 자르기
    short_clip = clip.subclip(start_time, end_time)
    # 새로운 파일로 저장
    short_clip.write_videofile(output_path, codec="libx264", fps=fps)


## 메인

In [ ]:
def main():
   # 1. 유튜브 mp3 다운로드
    url = "https://youtu.be/fre34cEeAYs?si=2JjkPcHBNmXsHurL"
    mp3_path = "youtube_output"
    mp4_path = "youtube_output"
    url_to_mp4(url, mp4_path)
    url_to_mp3(url, mp3_path)

    # 실제 mp3는 위에서 지정한 이름으로 저장됨
    mp3_file_path = "youtube_output.mp3"

    full_text_path="full_transcription.txt"

    # 2. Whisper로 텍스트 변환
    mp3_to_text(mp3_file_path, api_key)

    # 3. GPT로 쇼츠 포인트 추출
    text_to_shorts_points(full_text_path, api_key)

    # 4. 세그먼트 매칭
    segments_file = 'whisper_response.json'
    shorts_file = 'shorts_points.txt'
    output_file = 'result_by_segment.txt'

    segments = read_segments(segments_file)
    shorts_points = read_shorts_points(shorts_file)
    matched = match_shorts_to_segments(shorts_points, segments,  threshold=0.6, window_size=5)
    write_output(output_file, matched)

    print("🧩 세그먼트 매칭 및 저장 완료")

    # 5. 영상에서 쇼츠 자르기
    video_file = 'youtube_output.mp4'  # ⚠️ 직접 편집한 영상 위치로 설정 필요
    output_folder = 'output_clips\\'

    shorts_info = load_timestamps_from_file(output_file)

    fps = get_video_fps(video_file)

    for idx, (text, start_time, end_time) in enumerate(shorts_info):
        if start_time is not None and end_time is not None:
            output_path = f"{output_folder}short_clip_{idx + 1}.mp4"
            cut_and_save_video(video_file, start_time, end_time, output_path,fps)
            print(f"🎬 쇼츠 {idx + 1} 저장 완료: {output_path}")
        else:
            print(f"⚠️ 쇼츠 {idx + 1}는 매칭 실패로 저장되지 않았습니다.")

    print("✅ 전체 작업 완료!")

if __name__ == '__main__':
    main()


